In [27]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
from konlpy.tag import Komoran
from tqdm.notebook import tqdm
import emoji
import gensim
import glob

from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
import wordcloud
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from matplotlib import font_manager

from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

# 데이터 읽기

In [2]:
path = "D:/멀티캠퍼스/비타민_프로젝트/final_data/"

In [3]:
file_list = glob.glob(path + "*.csv")  # .csv 파일만 가져옴

# 파일명만 추출
file_list = [file.split("\\")[-1] for file in file_list]

In [4]:
for FL in file_list :
    df = pd.read_csv(path + FL, on_bad_lines="skip")
    df[["year","month", "day", "no"]] = df["date"].str.split(".", expand=True)
    df = df.drop("no", axis= 1)
    df["year"] = "20" + df["year"]
    df["date_full"] =  df["year"] + "-" + df["month"] + "-" + df["day"]
    df["date_full"] = pd.to_datetime(df["date_full"], format="%Y-%m-%d")

#### 이모지 없애기

In [5]:
def remove_emoji(df):

    for i in tqdm(df.index):
      df.loc[i, 'review'] = emoji.replace_emoji(df.loc[i, 'review'], replace='')
            
    return df

In [6]:
remove_emoji(df)

  0%|          | 0/6711 [00:00<?, ?it/s]

,top,url,brand,page,vitamin_title,date,sex,score,review,year,month,day,date_full
0,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,1,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,24.08.08.,NaN,5,칼배송 굿\n요오드 없는 멀티비타민 필요했는데 드디어 찾았어요 ㅎㅎ,2024,08,08,2024-08-08
1,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,1,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,24.08.07.,NaN,5,재구매에요. 성분이나 가격이 만족스러워요.,2024,08,07,2024-08-07
2,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,1,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,24.08.07.,여성,5,여성용 종합비타민이라 꾸준히 먹어보려구요~,2024,08,07,2024-08-07
3,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,1,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,24.07.30.,여성,5,배송도 빠르고 제품도 가성비가 좋네요,2024,07,30,2024-07-30
4,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,1,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,24.07.26.,NaN,5,영양분이 다 들어있어서 좋고 하루에 이거 하나만 먹으면 되니까 편해요 !,2024,07,26,2024-07-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6706,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,336,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,21.03.19.,NaN,5,1일 1정 간편한 섭취,2021,03,19,2021-03-19
6707,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,336,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,21.03.25.,NaN,4,조아요~~~~~~~~~~,2021,03,25,2021-03-25
6708,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,336,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,21.03.06.,NaN,5,잘 먹을께요~유한이라서 좋아요,2021,03,06,2021-03-06
6709,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,336,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,21.03.22.,NaN,5,조아요조아요조아요조아요,2021,03,22,2021-03-22


# Token

In [9]:
komoran = Komoran(userdic="D:/멀티캠퍼스/비타민_프로젝트/user_1.dic")
komoran

In [10]:
def extract_nouns(sentence):
  try:
    tokens = komoran.pos(sentence)
    nouns = [x[0] for x in tokens if x[1] in ("NNG", "NNP")]
    return nouns
  except Exception as e:
    print(e, sentence)
    return []

In [11]:
tqdm.pandas()
df["nouns"] = df["review"].progress_apply(lambda x: extract_nouns(x))
# df.to_csv("./top1.csv", encoding="utf-8-sig", index=False)

  0%|          | 0/6711 [00:00<?, ?it/s]

In [12]:
df

,top,url,brand,page,vitamin_title,date,sex,score,review,year,month,day,date_full,nouns
0,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,1,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,24.08.08.,NaN,5,칼배송 굿\n요오드 없는 멀티비타민 필요했는데 드디어 찾았어요 ㅎㅎ,2024,08,08,2024-08-08,"[칼, 배송, 굿, 오드, 멀티비타민, 필요]"
1,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,1,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,24.08.07.,NaN,5,재구매에요. 성분이나 가격이 만족스러워요.,2024,08,07,2024-08-07,"[재구매, 성분, 가격, 만족]"
2,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,1,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,24.08.07.,여성,5,여성용 종합비타민이라 꾸준히 먹어보려구요~,2024,08,07,2024-08-07,"[여성, 종합비타민]"
3,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,1,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,24.07.30.,여성,5,배송도 빠르고 제품도 가성비가 좋네요,2024,07,30,2024-07-30,"[배송, 제품, 가성비]"
4,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,1,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,24.07.26.,NaN,5,영양분이 다 들어있어서 좋고 하루에 이거 하나만 먹으면 되니까 편해요 !,2024,07,26,2024-07-26,"[영양분, 하루]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6706,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,336,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,21.03.19.,NaN,5,1일 1정 간편한 섭취,2021,03,19,2021-03-19,"[정, 섭취]"
6707,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,336,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,21.03.25.,NaN,4,조아요~~~~~~~~~~,2021,03,25,2021-03-25,"[아, 요]"
6708,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,336,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,21.03.06.,NaN,5,잘 먹을께요~유한이라서 좋아요,2021,03,06,2021-03-06,"[요, 유한]"
6709,20,https://brand.naver.com/yuhan/products/5411758...,유한양행,336,센스밸런스 여성 멀티비타민 3개월분 (84정) 유한양행 여성용 비타민,21.03.22.,NaN,5,조아요조아요조아요조아요,2021,03,22,2021-03-22,"[아, 요조, 아, 요조, 아, 요조, 아]"


## 명사만 집계하기

In [13]:
tqdm.pandas()
# 'nouns' 열의 각 항목을 문자열로 변환
df['nouns'] = df['nouns'].progress_apply(lambda x: str(x) if isinstance(x, list) else x)

  0%|          | 0/6711 [00:00<?, ?it/s]

In [14]:
tqdm.pandas()
df["nouns"] = df["nouns"].progress_apply(lambda x: eval(str(x)))

  0%|          | 0/6711 [00:00<?, ?it/s]

In [15]:
df.iloc[0]["nouns"]

['칼', '배송', '굿', '오드', '멀티비타민', '필요']

# scikit-learn TfidfVectorize

In [16]:
def dummy_func(doc):
    return doc
    
tfidf = TfidfVectorizer(
    analyzer = "word",
    tokenizer = dummy_func,
    preprocessor = dummy_func,
    token_pattern= None
)

In [17]:
tfidf_csr_matrix = tfidf.fit_transform(df["nouns"])

In [18]:
tfidf_csr_matrix, type(tfidf_csr_matrix)

(<6711x2320 sparse matrix of type '<class 'numpy.float64'>'
 	with 28288 stored elements in Compressed Sparse Row format>,
 scipy.sparse._csr.csr_matrix)

# CSR matrix

In [19]:
tfidf_vector_list = tfidf_csr_matrix.toarray()
sample = tfidf_vector_list
vocab = tfidf.vocabulary_  # 각 토큰별로 번호를 알려준다 

#### compressed sparse matrix를 살펴보면 각 단어의 인덱스와 tf-idf 스코어 값을 알 수 있음

In [21]:
# 희소 행렬을 밀집 배열로 변환
tfidf_vector_list = tfidf_csr_matrix.toarray()

# 단어 인덱스와 단어 매핑
vocab = tfidf.vocabulary_  # 각 단어에 대한 인덱스 사전

# 예시 출력
# print("문서 0의 단어 리스트:", df.iloc[0]["nouns"])
# print("문서 0의 TF-IDF 벡터:", tfidf_csr_matrix[0])
# print("단어와 인덱스 매핑:", vocab)

# 인덱스를 단어로 매핑
index_to_word = {v: k for k, v in vocab.items()}
# sample_vector = tfidf_vector_list[0]

sample_vector = tfidf_vector_list.sum(axis=0)

# 인덱스와 TF-IDF 값 함께 출력 (값이 오름차순으로 정렬)
index_value_pairs = [(index, sample_vector[index]) for index in range(len(sample_vector)) if sample_vector[index] > 0]
sorted_index_value_pairs = sorted(index_value_pairs, key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력
for index, value in sorted_index_value_pairs:
    word = index_to_word.get(index, "Unknown")
    print(f"인덱스: (0, {index}) 단어: {word}, 값: {value}")

인덱스: (0, 753) 단어: 배송, 값: 373.3167434696414
인덱스: (0, 1244) 단어: 알, 값: 278.38721214338324
인덱스: (0, 224) 단어: 구매, 값: 265.9083250016916
인덱스: (0, 2297) 단어: 효과, 값: 250.59860032307063
인덱스: (0, 2117) 단어: 포장, 값: 222.43545152889368
인덱스: (0, 1820) 단어: 주문, 값: 193.8403921528283
인덱스: (0, 884) 단어: 비타민, 값: 193.63059966403134
인덱스: (0, 2030) 단어: 크기, 값: 190.60600728949308
인덱스: (0, 1381) 단어: 영양제, 값: 169.13044600930027
인덱스: (0, 1711) 단어: 재구매, 값: 166.44634043599407
인덱스: (0, 1786) 단어: 제품, 값: 151.33143852313336
인덱스: (0, 1252) 단어: 알약, 값: 150.64911483982794
인덱스: (0, 80) 단어: 감사, 값: 146.45543235842683
인덱스: (0, 593) 단어: 만족, 값: 139.94692570396086
인덱스: (0, 655) 단어: 목넘김, 값: 123.05840846973163
인덱스: (0, 817) 단어: 복용, 값: 117.73739562714415
인덱스: (0, 2142) 단어: 피곤, 값: 113.78674279122741
인덱스: (0, 415) 단어: 달, 값: 108.262138475887
인덱스: (0, 1006) 단어: 선물, 값: 105.99008957414387
인덱스: (0, 1341) 단어: 여성, 값: 105.20332469100943
인덱스: (0, 2166) 단어: 하루, 값: 104.19930050081425
인덱스: (0, 2104) 단어: 편리, 값: 103.24270541825278
인덱스: (0, 1527) 단어: 유한양

#### CSR matrix를 데이터 프레임으로 만들기

In [22]:
# 토큰을 직접 입력받아 처리하도록 dummy_func 정의
def dummy_func(doc):
    return doc # 여기서는 간단히 공백으로 분리된 단어 리스트를 반환

# TfidfVectorizer 설정
tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_func,
    preprocessor=lambda x: x,  # 토큰화된 리스트를 그대로 사용
    token_pattern=None  # 기본 토큰화 패턴 사용 안 함
)

# 벡터화 진행
tfidf_csr_matrix = tfidf.fit_transform(df["nouns"])

# 희소 행렬을 밀집 배열로 변환
tfidf_vector_list = tfidf_csr_matrix.toarray()

# 단어 인덱스와 단어 매핑
vocab = tfidf.vocabulary_  # 각 단어에 대한 인덱스 사전
index_to_word = {v: k for k, v in vocab.items()}

# 모든 문서의 TF-IDF 벡터를 합산
total_vector = tfidf_vector_list.sum(axis=0)

# 인덱스와 TF-IDF 값 함께 출력 (값이 내림차순으로 정렬)
index_value_pairs = [(index, total_vector[index]) for index in range(len(total_vector)) if total_vector[index] > 0]
sorted_index_value_pairs = sorted(index_value_pairs, key=lambda x: x[1], reverse=True)

# 데이터프레임으로 변환
df_result = pd.DataFrame(sorted_index_value_pairs, columns=["단어 인덱스", "총 TF-IDF 값"])
df_result["단어"] = df_result["단어 인덱스"].apply(lambda x: index_to_word.get(x, "Unknown"))
df_result = df_result[["단어", "단어 인덱스", "총 TF-IDF 값"]]

# 결과 출력
df_result

,단어,단어 인덱스,총 TF-IDF 값
0,배송,753,373.316743
1,알,1244,278.387212
2,구매,224,265.908325
3,효과,2297,250.598600
4,포장,2117,222.435452
...,...,...,...
2315,의,1547,0.063763
2316,익,1612,0.063763
2317,조직,1797,0.063763
2318,친정,1991,0.063763


# Gensim을 이용한 word2vec 모델 학습

In [23]:
df_gensim = pd.DataFrame(df["nouns"])
df_gensim = df_gensim.rename(columns={"nouns":"tokens"})
df_gensim

,tokens
0,"[칼, 배송, 굿, 오드, 멀티비타민, 필요]"
1,"[재구매, 성분, 가격, 만족]"
2,"[여성, 종합비타민]"
3,"[배송, 제품, 가성비]"
4,"[영양분, 하루]"
...,...
6706,"[정, 섭취]"
6707,"[아, 요]"
6708,"[요, 유한]"
6709,"[아, 요조, 아, 요조, 아, 요조, 아]"


#### 문자열을 리스트로 변환해주기

In [24]:
tqdm.pandas()
df_gensim["tokens"] = df_gensim["tokens"].progress_apply(lambda x : eval(str(x)))

  0%|          | 0/6711 [00:00<?, ?it/s]

### gensim word2vec 학습

In [28]:
loss_list = []

class MonitorCallback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 1
        self.prev_loss = 0
        loss_list = []
        
    def on_epoch_end(self, model):
        cum_loss = model.get_latest_training_loss()
        current_loss = cum_loss - self.prev_loss
        loss_list.append(current_loss)
        print(f"epoch: {self.epoch} loss: {current_loss}") 
        self.prev_loss = cum_loss
        self.epoch += 1

In [29]:
model = Word2Vec(
    epochs=5,
    sentences=df_gensim["tokens"],
    vector_size=32,
    window=10,
    min_count=30,
    workers=4,
    sg=1,
    callbacks=[MonitorCallback()],
    compute_loss=True
)

epoch: 1 loss: 30691.171875
epoch: 2 loss: 30953.453125
epoch: 3 loss: 29081.828125
epoch: 4 loss: 28169.0625
epoch: 5 loss: 28856.03125


In [30]:
model.wv["비타민"]

array([-0.19951512, -0.2499393 ,  0.20031154,  0.16145231, -0.0330874 ,
        0.01323292,  0.15859608,  0.07890945,  0.06340899, -0.03633522,
        0.22211167, -0.4449486 , -0.20328905, -0.34302026,  0.015239  ,
       -0.24587147, -0.07339524,  0.2500015 ,  0.02806889,  0.31061685,
        0.11637247,  0.42232966,  0.26114655,  0.22841972,  0.03514044,
        0.17642152, -0.31304422,  0.008561  ,  0.20417042, -0.28886896,
       -0.13525136,  0.29949972], dtype=float32)

In [31]:
model.wv.most_similar("비타민")

[('멀티비타민', 0.9912785291671753),
 ('도움', 0.9900797605514526),
 ('기존', 0.9879265427589417),
 ('종합비타민', 0.9875350594520569),
 ('성분', 0.9872303009033203),
 ('이것저것', 0.9869367480278015),
 ('티', 0.9868425726890564),
 ('한', 0.9859538078308105),
 ('섭취', 0.9856523871421814),
 ('선택', 0.9855057597160339)]

In [32]:
model.save("./word2vec_model_1.model")

In [33]:
loaded_model = Word2Vec.load("word2vec_model_1.model")

In [35]:
model.wv.most_similar("유한양행")

[('가성비', 0.990895688533783),
 ('정', 0.9894336462020874),
 ('이것저것', 0.9894261360168457),
 ('처음', 0.9890882968902588),
 ('예정', 0.9890086650848389),
 ('검색', 0.9887050986289978),
 ('피로', 0.9884032011032104),
 ('섭취', 0.9881317019462585),
 ('중', 0.987533688545227),
 ('이', 0.9875297546386719)]